In [9]:
!pip install numpy pandas yfinance gunicorn lxml scikit-learn dash plotly


In [10]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px
from dash import dcc, html
from dash.dependencies import Input, Output, State
import dash
import plotly.graph_objs as go
from datetime import datetime as dt
from sklearn.svm import SVR


In [11]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Div([ # Stock code input
        html.Label("Enter Stock Code:"),
        dcc.Input(id='stock-code', type='text', value='AAPL'),
        html.Button('Submit', id='stock-button')
    ]),
    html.Div([ # Date range picker input
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-picker',
            start_date='2023-01-01',
            end_date='2023-10-01'
        )
    ]),
    html.Div([ # Stock price button, Indicators button, Number of days of forecast input, Forecast button
        html.Button('Fetch Stock Price', id='price-button'),
        html.Button('Fetch Indicators', id='indicator-button'),
        html.Label("Number of Days for Forecast:"),
        dcc.Input(id='forecast-days', type='number', value=7),
        html.Button('Forecast', id='forecast-button')
    ]),
    html.Div(id='stock-info'), # Stock information output
    html.Div(id='stock-graph'), # Stock price graph output
    html.Div(id='indicator-graph'), # Indicator graph output
    html.Div(id='forecast-graph') # Forecast graph output
])


In [12]:
@app.callback(Output('stock-info', 'children'),
              [Input('stock-button', 'n_clicks')],
              [State('stock-code', 'value')])
def update_stock_info(n_clicks, stock_code):
    ticker = yf.Ticker(stock_code)
    info = ticker.info
    df = pd.DataFrame().from_dict(info, orient="index").T
    return f"Company Name: {df['longName'].values[0]} | Logo: {df['logo_url'].values[0]} | Description: {df['longBusinessSummary'].values[0]}"


In [13]:
@app.callback(Output('stock-graph', 'children'),
              [Input('price-button', 'n_clicks')],
              [State('stock-code', 'value'),
               State('date-picker', 'start_date'),
               State('date-picker', 'end_date')])
def update_stock_graph(n_clicks, stock_code, start_date, end_date):
    df = yf.download(stock_code, start=start_date, end=end_date)
    fig = px.line(df, x=df.index, y=['Open', 'Close'], title="Stock Price vs Date")
    return dcc.Graph(figure=fig)


In [14]:
@app.callback(Output('forecast-graph', 'children'),
              [Input('forecast-button', 'n_clicks')],
              [State('stock-code', 'value'),
               State('forecast-days', 'value')])
def update_forecast_graph(n_clicks, stock_code, forecast_days):
    df = yf.download(stock_code, period='60d')
    dates = np.array(range(len(df)))
    dates = dates.reshape(-1, 1)
    close_prices = df['Close'].values

    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    svr_rbf.fit(dates, close_prices)

    forecast_dates = np.array(range(len(df), len(df) + forecast_days))
    forecast_dates = forecast_dates.reshape(-1, 1)
    forecast = svr_rbf.predict(forecast_dates)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df.index, y=close_prices, mode='lines', name='Historical Data'))
    fig.add_trace(go.Scatter(x=pd.date_range(df.index[-1], periods=forecast_days), y=forecast, mode='lines', name='Forecast'))
    fig.update_layout(title='Stock Price Forecast', xaxis_title='Date', yaxis_title='Price')
    return dcc.Graph(figure=fig)


In [15]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8051, mode="inline")
    


[*********************100%%**********************]  1 of 1 completed

---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
File ~\anaconda3\lib\site-packages\yfinance\ticker.py:142, in Ticker.info(self=yfinance.Ticker object <AAPL>)
    140 @property
    141 def info(self) -> dict:
--> 142     return self.get_info()
        self = yfinance.Ticker object <AAPL>

File ~\anaconda3\lib\site-packages\yfinance\base.py:1736, in TickerBase.get_info(
    self=yfinance.Ticker object <AAPL>,
    proxy=None
)
   1734 def get_info(self, proxy=None) -> dict:
   1735     self._quote.proxy = proxy
-> 1736     data = self._quote.info
        self._quote = <yfinance.scrapers.quote.Quote object at 0x000001C3B1AF28C0>
        self = yfinance.Ticker object <AAPL>
   1737     return data

File ~\anaconda3\lib\site-packages\yfinance\scrapers\quote.py:571, in Quote.info(
    self=<yfinance.scrapers.quote.Quote o